https://drive.google.com/drive/folders/1peThKQigJkn6CTFpIfI3jfkgH5s65EeE -- данные

Используйте реализацию многослойного персептрона из ЛР 4. Реализуйте сверточный слой (прямое и обратное распространение). Соберите сверточную сеть. Попробуйте обучить классификатор кошек и собак с использованием собственной реализации сверточной нейронной сети. Вам также потребуется реализовать слой для преобразования многомерных массивов данных в одномерные.

Надо сделать:
1. Переписать персептрон (полностью);
2. Написать Адама;
3. Написать сверточный слой (прямое и обратное распространение);
4. Написать слой для стягивания тензора в вектор;
5. Обучить классификатор кошек и собак на своей нейронке;
6. *Написать пулинговый слой 

In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import classification_report

# 1. Решение задачи классификации кошек и собак

## 1.1 Загрузка датасета и аугментация

In [4]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../data/cats&dogs", 
    validation_split=0.2, 
    subset="both", 
    seed=13, 
    image_size=(200, 200), # СНАЧАЛА ВЫСОТА, ПОТОМ ШИРИНА
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Using 400 files for validation.


In [5]:
data_augmentation_flip = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
])

In [6]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation_flip(img), label)
)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


![cat](..\memes\0_yV46bI9fS12FXrh-.png)

## 1.2 Построение модели

In [7]:
inputs = tf.keras.Input(shape=(200, 200, 3))

x = tf.keras.layers.Rescaling(1. / 255)(inputs)

x = tf.keras.layers.Conv2D(64, 3, strides=2, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

x = tf.keras.layers.Conv2D(128, 4, strides=2, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

classification_model = tf.keras.Model(inputs, outputs)

In [8]:
classification_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [9]:
classification_model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
50/50 [==============================] - 13s 246ms/step - loss: 0.8359 - accuracy: 0.5281 - val_loss: 0.6796 - val_accuracy: 0.5450
Epoch 2/5
50/50 [==============================] - 12s 232ms/step - loss: 0.6632 - accuracy: 0.5969 - val_loss: 0.6460 - val_accuracy: 0.6350
Epoch 3/5
50/50 [==============================] - 12s 235ms/step - loss: 0.6212 - accuracy: 0.6669 - val_loss: 0.6336 - val_accuracy: 0.6175
Epoch 4/5
50/50 [==============================] - 12s 234ms/step - loss: 0.6245 - accuracy: 0.6594 - val_loss: 0.6170 - val_accuracy: 0.6600
Epoch 5/5
50/50 [==============================] - 12s 233ms/step - loss: 0.6064 - accuracy: 0.6619 - val_loss: 0.6304 - val_accuracy: 0.6750


In [10]:
img = tf.keras.preprocessing.image.load_img(
    "../data/cats&dogs/cats/2.jpg", target_size=(200, 200)
)

img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

print("Это собака" if np.argmax(classification_model.predict(img_array)) else "Это кот")

1/1 [==============================] - 0s 85ms/step
Это кот


In [11]:
pred = [np.argmax(pred) for pred in classification_model.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

13/13 [==============================] - 1s 56ms/step
              precision    recall  f1-score   support

         0.0       0.66      0.67      0.66       191
         1.0       0.69      0.68      0.69       209

    accuracy                           0.68       400
   macro avg       0.67      0.67      0.67       400
weighted avg       0.68      0.68      0.68       400



# 2. Работа с уже обученными моделями

## 2.1 Загрузка моделей InceptionV3 и VGG19

In [12]:
vgg = tf.keras.applications.VGG19(weights='imagenet', include_top=True)
# vgg.summary()

In [13]:
iv3 = tf.keras.applications.InceptionV3(weights='imagenet', include_top=True)
# iv3.summary()

## 2.2 Feature-extraction и Fine-tuning для VGG19

In [14]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../data/objects", 
    validation_split=0.2, 
    subset="both", 
    seed=9, 
    image_size=(224, 224),
    batch_size=32
)

Found 216 files belonging to 3 classes.
Using 173 files for training.
Using 43 files for validation.


In [15]:
input = vgg.input
my_layer = tf.keras.layers.Dense(3, activation='softmax')
output = my_layer(vgg.layers[-2].output)

my_vgg = tf.keras.Model(input, output)

In [16]:
pred = [np.argmax(pred) for pred in my_vgg.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 4s 1s/step
              precision    recall  f1-score   support

         0.0       0.46      0.40      0.43        15
         1.0       0.20      0.21      0.21        14
         2.0       0.27      0.29      0.28        14

    accuracy                           0.30        43
   macro avg       0.31      0.30      0.30        43
weighted avg       0.31      0.30      0.31        43



In [17]:
my_vgg.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
for layer in my_vgg.layers:
    layer.trainable = False
my_vgg.layers[-1].trainable = True

In [19]:
my_vgg.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
6/6 [==============================] - 81s 13s/step - loss: 1603.8438 - accuracy: 0.2717 - val_loss: 9.8863 - val_accuracy: 0.3488
Epoch 2/5
6/6 [==============================] - 76s 13s/step - loss: 19.2764 - accuracy: 0.4220 - val_loss: 2.7130 - val_accuracy: 0.2326
Epoch 3/5
6/6 [==============================] - 77s 13s/step - loss: 6.0088 - accuracy: 0.3757 - val_loss: 1.2099 - val_accuracy: 0.3953
Epoch 4/5
6/6 [==============================] - 77s 13s/step - loss: 9.1956 - accuracy: 0.3757 - val_loss: 7.9244 - val_accuracy: 0.3488
Epoch 5/5
6/6 [==============================] - 77s 13s/step - loss: 5.1251 - accuracy: 0.3699 - val_loss: 3.3204 - val_accuracy: 0.3256


In [20]:
pred = [np.argmax(pred) for pred in my_vgg.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 4s 1s/step
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        15
         1.0       0.33      1.00      0.49        14
         2.0       0.00      0.00      0.00        14

    accuracy                           0.33        43
   macro avg       0.11      0.33      0.16        43
weighted avg       0.11      0.33      0.16        43



c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
my_vgg.layers[-2].trainable = True

In [22]:
my_vgg.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
6/6 [==============================] - 78s 13s/step - loss: 2.4547 - accuracy: 0.3410 - val_loss: 1.2384 - val_accuracy: 0.3721
Epoch 2/5
6/6 [==============================] - 78s 13s/step - loss: 1.3153 - accuracy: 0.4798 - val_loss: 1.2136 - val_accuracy: 0.3256
Epoch 3/5
6/6 [==============================] - 77s 13s/step - loss: 1.1581 - accuracy: 0.3873 - val_loss: 2.5256 - val_accuracy: 0.3488
Epoch 4/5
6/6 [==============================] - 76s 13s/step - loss: 1.5862 - accuracy: 0.4509 - val_loss: 1.2119 - val_accuracy: 0.3488
Epoch 5/5
6/6 [==============================] - 76s 12s/step - loss: 1.2265 - accuracy: 0.4624 - val_loss: 1.0899 - val_accuracy: 0.3488


In [23]:
pred = [np.argmax(pred) for pred in my_vgg.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 4s 962ms/step
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        15
         1.0       0.50      0.07      0.12        14
         2.0       0.34      1.00      0.51        14

    accuracy                           0.35        43
   macro avg       0.28      0.36      0.21        43
weighted avg       0.27      0.35      0.21        43



c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2.3 Feature-extraction и Fine-tuning для InceptionV3

In [24]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../data/objects", 
    validation_split=0.2, 
    subset="both", 
    seed=9, 
    image_size=(299, 299),
    batch_size=32
)

Found 216 files belonging to 3 classes.
Using 173 files for training.
Using 43 files for validation.


In [25]:
input = iv3.input
my_layer = tf.keras.layers.Dense(3, activation='softmax')
output = my_layer(iv3.layers[-2].output)

my_iv3 = tf.keras.Model(input, output)

In [26]:
pred = [np.argmax(pred) for pred in my_iv3.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 3s 390ms/step
              precision    recall  f1-score   support

         0.0       0.45      0.67      0.54        15
         1.0       0.00      0.00      0.00        14
         2.0       0.48      0.71      0.57        14

    accuracy                           0.47        43
   macro avg       0.31      0.46      0.37        43
weighted avg       0.31      0.47      0.37        43



c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
my_iv3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
for layer in my_iv3.layers:
    layer.trainable = False
my_iv3.layers[-1].trainable = True

In [29]:
my_iv3.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
6/6 [==============================] - 58s 7s/step - loss: 0.2616 - accuracy: 0.8902 - val_loss: 25.2841 - val_accuracy: 0.3256
Epoch 2/5
6/6 [==============================] - 42s 7s/step - loss: 0.1418 - accuracy: 0.9711 - val_loss: 817.0413 - val_accuracy: 0.3256
Epoch 3/5
6/6 [==============================] - 43s 7s/step - loss: 0.1597 - accuracy: 0.9653 - val_loss: 5190.0977 - val_accuracy: 0.3256
Epoch 4/5
6/6 [==============================] - 42s 7s/step - loss: 0.1379 - accuracy: 0.9653 - val_loss: 54033.5391 - val_accuracy: 0.3256
Epoch 5/5
6/6 [==============================] - 43s 7s/step - loss: 0.0974 - accuracy: 0.9711 - val_loss: 69454.8047 - val_accuracy: 0.3256


In [30]:
pred = [np.argmax(pred) for pred in my_iv3.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 3s 401ms/step
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        15
         1.0       0.33      1.00      0.49        14
         2.0       0.00      0.00      0.00        14

    accuracy                           0.33        43
   macro avg       0.11      0.33      0.16        43
weighted avg       0.11      0.33      0.16        43



c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
